**INFO7390 - ASSIGNMENT 2 - Crash Course in Generative AI Worked Examples**

Amrutha Edara -002776773

Niharika Santhoshini Karri - 002727629

##GenerativeAI use case with Dialgoue Summarization



###Contents:

* Set up Kernel and Required Dependencies

* Summarize Dialogue without Prompt Engineering

* Summarize Dialogue with an Instruction Prompt

 --Zero Shot Inference with an Instruction Prompt

 --Zero Shot Inference with the Prompt Template from FLAN-T5

* Summarize Dialogue with One Shot and Few Shot Inference

  -- One Shot Inference

  -- Few Shot Inference

* Generative Configuration Parameters for Inference

In [ ]:
!pip install --upgrade pip
!pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

!pip install \
    transformers==4.27.2 \
    datasets==2.11.0  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.6 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 101.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.13.1 which is incompatible.
torchtext 0.16.0 re

In [ ]:
#To load dataset and some of the models and tokenizer needed to perform summarization


from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

###Summarize data without prompt engineering

In [ ]:
huggingface_dataset_name = "TanveerAman/AMI-Corpus-Text-Summarization"

dataset = load_dataset(huggingface_dataset_name)

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset['ID'] = range(len(dataset))


In [ ]:
example_indices = [4, 6]

dash_line = '-'.join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['Dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['Summaries'])
    print(dash_line)
    print()

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
D: Great man .
D: Who starts ?
A: Well I'll uh start just with another presentation , so then we can uh look at th at the agenda uh for this meeting .
D: Alright , great .
D: Alright .
A: Okay .
A: I've put some uh new things in the in the map .
D: Mm-hmm .
A: Uh oh .
A: This is it .
A: I don't know the shortcut , so Ah F_ five .
A: Well our functional design meeting , that's the stage we're in .
C: Mm .
A: And you also ha all three of you have uh prepared something about it .
D: Yes .
A: Well um in we'll uh just have a look at the at the notes from the previous meeting , what we uh thought we had dec decided .
A: But uh Uh then we'll uh look at uh the three uh presentations uh from you .
D: 'Kay .
A: I think you have prepared uh all three uh ?
C: Well , yeah .
D

#### Load the FLAN-T5 model, creating an instance of the AutoModelForSeq2SeqLM class with the .from_pretrained() method.

In [ ]:
#Loading Flan-T5 model to compare the human base summary to the one generated by the Model

model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

####To perform encoding and decoding, you need to work with text in a tokenized form. Download the tokenizer for FLAN-T5 model, using AutoTokenizer.from_pretrained() method.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
#testing the tokenizer with an example
sentence = "How are you, Harry?"

sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )

print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)

ENCODED SENTENCE:
tensor([ 571,   33,   25,    6, 8929,   58,    1])

DECODED SENTENCE:
How are you, Harry?


In [ ]:
from transformers import pipeline
from transformers import BartTokenizer, BartForConditionalGeneration

# Load the summarization model and tokenizer
summarization_model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
summarization_tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Create a summarization pipeline
summarize = pipeline("summarization", model=summarization_model, tokenizer=summarization_tokenizer)

for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['Dialogue']
    summary = dataset['test'][index]['Summaries']

    # Truncate the dialogue if it's too long
    tokens = summarization_tokenizer(dialogue, truncation=True, max_length=1024, return_tensors='pt')
    truncated_dialogue = summarization_tokenizer.decode(tokens.input_ids[0], skip_special_tokens=True)

    # Summarize the truncated dialogue
    summarized_dialogue = summarize(truncated_dialogue, max_length=512, min_length=100, do_sample=False)[0]['summary_text']

    # Tokenize the summarized dialogue for the original model
    inputs = tokenizer(summarized_dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=10,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'SUMMARIZED INPUT:\n{summarized_dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
D: Great man .
D: Who starts ?
A: Well I'll uh start just with another presentation , so then we can uh look at th at the agenda uh for this meeting .
D: Alright , great .
D: Alright .
A: Okay .
A: I've put some uh new things in the in the map .
D: Mm-hmm .
A: Uh oh .
A: This is it .
A: I don't know the shortcut , so Ah F_ five .
A: Well our functional design meeting , that's the stage we're in .
C: Mm .
A: And you also ha all three of you have uh prepared something about it .
D: Yes .
A: Well um in we'll uh just have a look at the at the notes from the previous meeting , what we uh thought we had dec decided .
A: But uh Uh then we'll uh look at uh the three uh presentations uh from you .
D: 'Kay .
A: I think you have prepared uh all three uh ?
C: Well , yeah .
D: 

In the above code snippet, we are using a pre-trained transformer model- BART and its tokenizer for text summarization, to limit the max token limit within the range of < 512 tokens of the model to ease further analsysis done using FLAN-T5.

### **Summarize Dialogue with an Instruction Prompt**

####  Zero Shot Inference with an Instruction Prompt

To instruct the model to perform a task - summarize a dialogue - you can take the dialogue and convert it into an instruction prompt. This is often called **zero shot inference**.

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['Dialogue']
    summary = dataset['test'][index]['Summaries']

    # Construct the prompt
    prompt = f"""
    Summarize the following conversation.

    {dialogue}

    Summary:
    """

    # Truncate the dialogue if it's too long
    truncated_dialogue = summarization_tokenizer.encode(dialogue, truncation=True, max_length=512 - 50, return_tensors='pt')
    truncated_dialogue = summarization_tokenizer.decode(truncated_dialogue[0], skip_special_tokens=True)

    # Reconstruct the prompt with the truncated dialogue
    prompt = f"""
    Summarize the following conversation.

    {truncated_dialogue}

    Summary:
    """

    # Input constructed prompt to the model
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')


Token indices sequence length is longer than the specified maximum sequence length for this model (530 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

    Summarize the following conversation.
    
    D: Great man.
D: Who starts?
A: Well I'll uh start just with another presentation, so then we can uh look at th at the agenda uh for this meeting.
D: Alright, great.
D: Alright.
A: Okay.
A: I've put some uh new things in the in the map.
D: Mm-hmm.
A: Uh oh.
A: This is it.
A: I don't know the shortcut, so Ah F_ five.
A: Well our functional design meeting, that's the stage we're in.
C: Mm.
A: And you also ha all three of you have uh prepared something about it.
D: Yes.
A: Well um in we'll uh just have a look at the at the notes from the previous meeting, what we uh thought we had dec decided.
A: But uh Uh then we'll uh look at uh the three uh presentations uh from you.
D: 'Kay.
A: I think you have prepared uh all th

#### Zero Shot Inference with the Prompt Template from FLAN-T5

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = dataset['test'][index]['Dialogue']
    summary = dataset['test'][index]['Summaries']

    # Truncate the dialogue to fit within the model's maximum sequence length
    truncated_dialogue = tokenizer.encode(dialogue, add_special_tokens=False, max_length=450, truncation=True)
    truncated_dialogue = tokenizer.decode(truncated_dialogue)

    prompt = f"""
Dialogue:

{truncated_dialogue}

What was going on?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Dialogue:

D: Great man. D: Who starts? A: Well I'll uh start just with another presentation, so then we can uh look at th at the agenda uh for this meeting. D: Alright, great. D: Alright. A: Okay. A: I've put some uh new things in the in the map. D: Mm-hmm. A: Uh oh. A: This is it. A: I don't know the shortcut, so Ah F_ five. A: Well our functional design meeting, that's the stage we're in. C: Mm. A: And you also ha all three of you have uh prepared something about it. D: Yes. A: Well um in we'll uh just have a look at the at the notes from the previous meeting, what we uh thought we had dec decided. A: But uh Uh then we'll uh look at uh the three uh presentations uh from you. D: 'Kay. A: I think you have prepared uh all three uh? C: Well, yeah. D: Yeah. A: Um we

#### **Summarize Dialogue with One Shot and Few Shot Inference**

One shot and few shot inference are the practices of providing an LLM with either one or more full examples of prompt-response pairs that match your task - before your actual prompt that you want completed. This is called "in-context learning" and puts your model into a state that understands your specific task.

In [ ]:
from transformers import BartTokenizer

def make_prompt(example_indices_full, example_index_to_summarize, max_length_per_segment=100):
    tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')
    prompt = ''

    for index in example_indices_full:
        dialogue = dataset['test'][index]['Dialogue']
        summary = dataset['test'][index]['Summaries']

        # Truncate the dialogue and summary if they are too long
        truncated_dialogue = tokenizer.decode(tokenizer.encode(dialogue, truncation=True, max_length=max_length_per_segment), skip_special_tokens=True)
        truncated_summary = tokenizer.decode(tokenizer.encode(summary, truncation=True, max_length=max_length_per_segment), skip_special_tokens=True)

        prompt += f"""
Dialogue:

{truncated_dialogue}

What was going on?
{truncated_summary}


"""

    dialogue = dataset['test'][example_index_to_summarize]['Dialogue']
    truncated_dialogue = tokenizer.decode(tokenizer.encode(dialogue, truncation=True, max_length=max_length_per_segment), skip_special_tokens=True)

    prompt += f"""
Dialogue:

{truncated_dialogue}

What was going on?
"""

    return prompt


In [ ]:
example_indices_full = [6]
example_index_to_summarize = 6

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)


Dialogue:

B: How do you wear this thing?
A: Hmm.
A: Mm mm mm.
B: Not too many cables and stuff.
B: Original.
A: Is recorded?
A: Okay?
A: Okay so welcome everyone.
A: So we are here for the kickoff meeting of uh the process of designing a new remote control.
A: So I will first start with a warm welcome opening stuff, then uh we will uh

What was going on?
The Project Manager introduced the project to the group and gave a timeline for the project.
The group trained themselves to use the meeting room tools by drawing on the whiteboard.
The Project Manager presented the project budget and the projected price point and profit goals.
The group discussed several of their initial ideas for the features of the product.
They discussed making the remote able to control multiple devices, protection from water or from dropping the remote, and a locator function.
The Project



Dialogue:

B: How do you wear this thing?
A: Hmm.
A: Mm mm mm.
B: Not too many cables and stuff.
B: Original.
A: Is recor

In [ ]:
summary = dataset['test'][example_index_to_summarize]['Summaries']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
The Project Manager introduced the project to the group and gave a timeline for the project.
The group trained themselves to use the meeting room tools by drawing on the whiteboard.
The Project Manager presented the project budget and the projected price point and profit goals.
The group discussed several of their initial ideas for the features of the product.
They discussed making the remote able to control multiple devices, protection from water or from dropping the remote, and a locator function.
The Project Manager then instructed the User Interface Designer to research users' requirements, and instructed the Industrial and User Interface Designers to research the functions and usability features that were discussed in the meeting.
The Industrial Designer will do research on the whistle tracking locator function and other functional components.
The User Interf

####Few Shot Inference

In [ ]:
example_indices_full = [4, 5, 6]
example_index_to_summarize = 6

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)


Dialogue:

D: Great man.
D: Who starts?
A: Well I'll uh start just with another presentation, so then we can uh look at th at the agenda uh for this meeting.
D: Alright, great.
D: Alright.
A: Okay.
A: I've put some uh new things in the in the map.
D: Mm-hmm.
A: Uh oh.
A: This is it.
A:

What was going on?
There are some new requirements for the project: the device will have to be for TV only, they should ignore the teletext function, as it is dated, and, also, the remote should be recognisable as a Real Reaction product.
The target group will be people below 30.
It transpires that customers are willing to pay more for fancier looking remotes, especially if they include LCD screens and speech recognition.
Functions like zapping are very prevalent, as well



Dialogue:

A:.
A: Okay.
C: Mm-hmm.
A: So we are here for the concept design meeting.
A: So, we will first start by summarizing the mm the previous meeting and the decision we've taken.
A: Uh I will take notes during this meeting so

we are passing this prompt to perform a few shot inference to get the summary for the 4th dialogue

In [ ]:
summary = dataset['test'][example_index_to_summarize]['Summaries']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
The Project Manager introduced the project to the group and gave a timeline for the project.
The group trained themselves to use the meeting room tools by drawing on the whiteboard.
The Project Manager presented the project budget and the projected price point and profit goals.
The group discussed several of their initial ideas for the features of the product.
They discussed making the remote able to control multiple devices, protection from water or from dropping the remote, and a locator function.
The Project Manager then instructed the User Interface Designer to research users' requirements, and instructed the Industrial and User Interface Designers to research the functions and usability features that were discussed in the meeting.
The Industrial Designer will do research on the whistle tracking locator function and other functional components.
The User Interf

####The difference we have added in this example, for performing the analysis using zero-shot, one-shot, and few-shot inference is that we have truncated the summaries before passing them into the model, so that they are not exceeding the 512 token limit for FLAN-T5 model.


* From the above, observations and our understanding, there is not much improvement from one-shot to few shot inference. It is concluded that the model is going to perform in similar manner after certain point say...5 shot inference or 6 shot inference.

###Generative Configuration Parameters for Inference

A convenient way of organizing the configuration parameters is to use GenerationConfig class.

In [ ]:
#generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
#generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
The group is going for a meeting to talk about their projects.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
The Project Manager introduced the project to the group and gave a timeline for the project.
The group trained themselves to use the meeting room tools by drawing on the whiteboard.
The Project Manager presented the project budget and the projected price point and profit goals.
The group discussed several of their initial ideas for the features of the product.
They discussed making the remote able to control multiple devices, protection from water or from dropping the remote, and a locator function.
The Project Manager then instructed the User Interface Designer to research users' requirements, and instructed the Industrial and User Interface Designers to research the functio

* Choosing max_new_tokens=10 will make the output text too short, so the dialogue summary will be cut. Putting do_sample = True and changing the temperature value you get more flexibility in the output.

REFERENCES:

https://www.coursera.org/learn/generative-ai-with-llms/lecture/wno7h/lab-1-walkthrough

https://serokell.io/blog/nn-and-one-shot-learning


https://huggingface.co/learn/nlp-course/chapter7/5

License
Copyright [2023]

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.